Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import time

In [2]:
#pickle_file = 'notMNIST.pickle'
pickle_file = 'notMNIST_sanitized.pickle'


with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (8912, 28, 28) (8912,)
Test set (8667, 28, 28) (8667,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (8912, 28, 28, 1) (8912, 10)
Test set (8667, 28, 28, 1) (8667, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [79]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

# Dropout probability for countering overfitting
dropout = [0.5, 0.5]

# exponential decay learning params
init_learn_rate = 0.1
decay_steps = 500
decay_rate = 0.98

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables - layer weights and biases.
  w1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  b1 = tf.Variable(tf.zeros([depth]))  
  w2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  b2 = tf.Variable(tf.constant(1.0, shape=[depth]))
  w3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  b3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  w4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  b4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, w1, [1, 1, 1, 1], padding='SAME')
    pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pooled + b1)
    
    conv = tf.nn.conv2d(tf.nn.dropout(hidden, dropout[0]), w2, [1, 1, 1, 1], padding='SAME')
    pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pooled + b2)
    
    shape = tf.nn.dropout(hidden, dropout[1]).get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, w3) + b3)
    return tf.matmul(hidden, w4) + b4
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.05, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [80]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  start = time.time()
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % (num_steps // 10) == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  
  print("Total loop time: <", int(time.time() - start), "secs> or <", int(time.time() - start)
      / 60, "mins and", int(time.time() - start) % 60, "secs>")

Initialized
Minibatch loss at step 0: 4.081130
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 100: 2.029756
Minibatch accuracy: 37.5%
Validation accuracy: 23.8%
Minibatch loss at step 200: 1.163866
Minibatch accuracy: 62.5%
Validation accuracy: 59.9%
Minibatch loss at step 300: 0.583212
Minibatch accuracy: 75.0%
Validation accuracy: 72.5%
Minibatch loss at step 400: 0.801797
Minibatch accuracy: 75.0%
Validation accuracy: 71.6%
Minibatch loss at step 500: 0.489807
Minibatch accuracy: 87.5%
Validation accuracy: 78.1%
Minibatch loss at step 600: 0.468563
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 700: 0.176205
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 800: 0.206363
Minibatch accuracy: 93.8%
Validation accuracy: 80.6%
Minibatch loss at step 900: 1.016083
Minibatch accuracy: 56.2%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.454136
Minibatch accuracy: 87.5%
Validation accuracy: 80.7

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a  
##### max pooling operation (`nn.max_pool()`)  
- stride = 2  
- kernel size 2.

---

#### Result:
See above code

---
Problem 2
---------

Try to get the best performance you can using a convolutional net.
a) Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture,  
b) Add Dropout, and/or  
c) Add learning rate decay.  

---

In [9]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 128

# exponential decay learning params
init_learn_rate = 0.1
decay_steps = 500
decay_rate = 0.96

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables - layer weights and biases.
  w1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  b1 = tf.Variable(tf.zeros([depth]))  
  w2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  b2 = tf.Variable(tf.constant(1.0, shape=[depth]))
  w3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  b3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  w4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  b4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob=1.0):
    conv = tf.nn.conv2d(data, w1, [1, 1, 1, 1], padding='SAME')
    pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pooled + b1)
    
    conv = tf.nn.conv2d(tf.nn.dropout(hidden, keep_prob), w2, [1, 1, 1, 1], padding='SAME')
    pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pooled + b2)
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, w3) + b3), keep_prob)
    return tf.matmul(hidden, w4) + b4
  
  # Training computation.
  logits = model(tf_train_dataset, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(init_learn_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  start = time.time()
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % (num_steps // 10) == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  
  print("Total loop time: <", int(time.time() - start), "secs> or <", int(time.time() - start)
      / 60, "mins and", int(time.time() - start) % 60, "secs>")

Initialized
Minibatch loss at step 0: 7.350584
Minibatch accuracy: 10.2%
Validation accuracy: 11.3%
Minibatch loss at step 300: 0.810053
Minibatch accuracy: 74.2%
Validation accuracy: 80.1%
Minibatch loss at step 600: 0.561247
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Minibatch loss at step 900: 0.698289
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Minibatch loss at step 1200: 0.616673
Minibatch accuracy: 79.7%
Validation accuracy: 84.5%
Minibatch loss at step 1500: 0.593611
Minibatch accuracy: 84.4%
Validation accuracy: 85.0%
Minibatch loss at step 1800: 0.498486
Minibatch accuracy: 82.8%
Validation accuracy: 85.5%
Minibatch loss at step 2100: 0.446924
Minibatch accuracy: 87.5%
Validation accuracy: 85.8%
Minibatch loss at step 2400: 0.492862
Minibatch accuracy: 85.2%
Validation accuracy: 86.2%
Minibatch loss at step 2700: 0.562496
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 3000: 0.546452
Minibatch accuracy: 84.4%
Validation accuracy:

# LeNet CNN (Too slow: ???)

In [5]:
image_size = 28 # W,H
batch_size = 16
patch_size = 5 # F
#depth = 16 # D
num_hidden = 16

# exponential decay learning params
init_learn_rate = 0.01
decay_steps = 500
decay_rate = 0.98

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables - layer weights and biases.
  w1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, 32], stddev=0.1))
  b1 = tf.Variable(tf.zeros([32]))  
  w2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, 32, 64], stddev=0.1))
  b2 = tf.Variable(tf.constant(0.1, shape=[64]))
  w3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * 64, num_hidden], stddev=0.1))
  b3 = tf.Variable(tf.constant(0.1, shape=[num_hidden]))
  w4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  b4 = tf.Variable(tf.constant(0.1, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob=1.0):
    conv = tf.nn.conv2d(data, w1, [1, 1, 1, 1], padding='SAME')
    relu = tf.nn.relu(conv + b1)
    pool = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(tf.nn.dropout(pool, keep_prob), w2, [1, 1, 1, 1], padding='SAME')
    relu = tf.nn.relu(conv + b2)
    pool = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, w3) + b3), keep_prob)
    return tf.matmul(hidden, w4) + b4
  
  # Training computation.
  logits = model(tf_train_dataset, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(init_learn_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 11

with tf.Session(graph=graph) as session:
  start = time.time()
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % (num_steps // 10) == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  
  print("Total loop time: <", int(time.time() - start), "secs> or <", int(time.time() - start)
      / 60, "mins and", int(time.time() - start) % 60, "secs>")

Initialized
Minibatch loss at step 0: 3.095192
Minibatch accuracy: 12.5%
Validation accuracy: 9.4%
Minibatch loss at step 1: 2.519120
Minibatch accuracy: 6.2%
Validation accuracy: 12.2%
Minibatch loss at step 2: 2.168639
Minibatch accuracy: 18.8%
Validation accuracy: 15.1%
Minibatch loss at step 3: 2.535597
Minibatch accuracy: 6.2%
Validation accuracy: 16.1%
Minibatch loss at step 4: 2.515879
Minibatch accuracy: 25.0%
Validation accuracy: 14.8%
Minibatch loss at step 5: 2.479053
Minibatch accuracy: 6.2%
Validation accuracy: 11.6%
Minibatch loss at step 6: 2.348463
Minibatch accuracy: 6.2%
Validation accuracy: 11.0%
Minibatch loss at step 7: 2.287859
Minibatch accuracy: 18.8%
Validation accuracy: 10.7%
Minibatch loss at step 8: 2.342532
Minibatch accuracy: 18.8%
Validation accuracy: 10.4%
Minibatch loss at step 9: 2.340959
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 10: 2.290623
Minibatch accuracy: 12.5%
Validation accuracy: 8.3%
Test accuracy: 7.4%
Total 